In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sn
from scipy import stats
from datetime import datetime
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/bike-sharing-demand/train.csv")
train_df.head()

In [ ]:
train_df.isna().sum()

In [ ]:
corrMatt = train_df[["temp","atemp","casual","registered","humidity","windspeed","count"]].corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sn.heatmap(corrMatt, mask=mask,vmax=.8, square=True,annot=True)

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3)
fig.set_size_inches(12, 5)
sn.regplot(x="temp", y="count", data=train_df,ax=ax1)
sn.regplot(x="windspeed", y="count", data=train_df,ax=ax2)
sn.regplot(x="humidity", y="count", data=train_df,ax=ax3)

In [ ]:
train_df = train_df.drop(["atemp"], axis=1)
train_df["date"] = pd.to_datetime(train_df["datetime"])
train_df["year"] = train_df["date"].dt.year
train_df["month"] = train_df["date"].dt.month
train_df["dyofweek"] = train_df["date"].dt.dayofweek
train_df["hour"] = train_df["date"].dt.hour

In [ ]:
train_df.drop(["datetime"],axis=1,inplace=True)

In [ ]:
train_df.info()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_df.drop(["registered","count","date","casual"],axis=1), train_df["count"], test_size=0.33, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import mean_squared_log_error,r2_score, mean_squared_error

RandomForest = RandomForestRegressor( random_state=42)
ExtraTrees = ExtraTreesRegressor(random_state=42)

y_train = np.log1p(y_train+1)
# Train the classifier
model = RandomForest.fit(X_train, y_train)
model2 = ExtraTrees.fit(X_train, y_train.ravel())


y_val_predicted = model.predict(X_val)
y_val_predicted2 = model2.predict(X_val)
y_val_predicted = np.exp(y_val_predicted)
y_val_predicted2 = np.expm1(y_val_predicted2)
print("The Score of the Regressor on the validation set is ", np.sqrt((mean_squared_log_error(y_val_predicted, y_val))))
print("The Score of the Regressor on the validation set is ", np.sqrt((mean_squared_log_error(y_val_predicted2, y_val))))

In [ ]:
test_df =pd.read_csv("../input/bike-sharing-demand/test.csv")
test_df = test_df.drop(["atemp"], axis=1)
test_df["date"] = pd.to_datetime(test_df["datetime"])
test_df["year"] = test_df["date"].dt.year
test_df["month"] = test_df["date"].dt.month
test_df["dyofweek"] = test_df["date"].dt.dayofweek
test_df["hour"] = test_df["date"].dt.hour

test_df.drop(["datetime",'date'],axis=1,inplace=True)
test_df.columns


In [ ]:
y_test_predicted2 = model2.predict(test_df)
y_test_predicted2 = np.expm1(y_test_predicted2)

In [ ]:
sub = pd.read_csv("../input/bike-sharing-demand/sampleSubmission.csv")
sub.head()

In [ ]:
sub["count"]= y_test_predicted2

In [ ]:
sub.to_csv("submission.csv", index=False)